# Why I like Julia

## 1. Julia is expressive and understandable
- In e.g. Python or R, libraries are mostly written in non-native language. [Here is an example with `pytorch`](https://github.com/pytorch/pytorch).


![](img/pytorch.png)


- Source code of most Julia libraries is written in **pure Julia**. [Here is an example with Flux.jl](https://github.com/FluxML/Flux.jl), the equivalent of `pytorch` in Julia.

  ![](img/flux.png)

- This allows to understand exactly what the functions you are using are doing ➡️ this makes you smarter

[Here is an example of how is implemented in a dense layer in Julia](https://github.com/FluxML/Flux.jl/blob/0038a60e266d0fca17aa8db99cd6453eb633ee7b/src/layers/basic.jl#L170)

```julia
function (a::Dense)(x::AbstractVecOrMat)
  _size_check(a, x, 1 => size(a.weight, 2))
  σ = NNlib.fast_act(a.σ, x)  # replaces tanh => tanh_fast, etc
  xT = _match_eltype(a, x)  # fixes Float64 input, etc.
  return σ.(a.weight * xT .+ a.bias)
end
```
- This allows you to easily contribute and develop to packages ➡️ this makes you more useful in this world


- The language enables you to express ideas in fewer lines of code than in traditional languages such as C++ or Fortran. Perhaps some of the best examples of this are from BeautifulAlgorithms.jl by Robert Moss. Below are some basic examples of loss functions in  Julia and how to program them in a single line of code.

![](https://github.com/mossr/BeautifulAlgorithms.jl/raw/master/img/png/loss_functions.png)

- The language is emoji friendly! This is how you would define a Lotka Volterra system in Julia

In [ ]:
function lotka_volterra!(du, u, p, t)
    α, β, γ, δ = p
    🐰, 🦊 = u
    d🐰, d🦊 = du
    d🐰 = α * 🐰 - β * 🦊 * 🐰
    d🦊 = γ*🐰 * 🦊  - δ * 🦊
end

## 1. Interactivity

Amazing IDE with VS code and inline prompts

![](https://code.visualstudio.com/assets/docs/languages/julia/overview.png)


## 2. Package management

Julia provides a **built-in package** manager called `Pkg` for **managing packages and environments**. Users can create a new environment and add specific packages to it, and each environment has its own set of dependencies. Julia also allows users to switch between different environments easily.

More on that later on!

# 3. Execution speed!

Let's construct a for loop summation of a random sequence of integers from 1 to 1,000,000,000 (1 billion) that are sampled without replacment.1 Here is the correct answer as a reference


In [1]:
using StatsBase, BenchmarkTools
n = 1_000_000_000;
function sum_n()
    s = 0
    for i in 1:n
        s = s + i
    end
    return s
end
@btime sum_n(samp)

In [ ]:
using RCall
R"""
n = 1000000000
sum_n = function(){
    s = 0
    for (i in n){
        s = s + i
    }
    s
}
print(system.time(x <- sum_n()))
"""

### Support for parallelism


```julia
Threads.@threads for i in 1:n
    myarray[1] = do_stuff(parameters[i])
end
```

## 3. Community well organized

There is an active community of users and developers who contribute high-quality packages regularly.

- Slack channel
- Discourse forum
- Youtube tutorials

Information available at https://julialang.org/community/

## 4. Multiple Dispatch



In [ ]:
abstract type PlantSpecies end

struct Oak <: PlantSpecies
    height::Float64
    leaf_area::Float64
end

struct Maple <: PlantSpecies
    height::Float64
    leaf_area::Float64
end

function aboveground_biomass(species::Oak)
    return 0.0314 * species.height^2.19 * species.leaf_area^0.91
end

function aboveground_biomass(species::Maple)
    return 0.0215 * species.height^2.42 * species.leaf_area^0.94
end


### More interesting use case of multiple dispatch: GPU computing

Multiple dispatch can be useful when it comes to GPU computing because it allows for efficient dispatch of functions to the GPU, which can lead to significant performance gains.

In GPU computing, data parallelism is often used to perform computations on large arrays in parallel. This involves splitting the input data into smaller chunks and executing the same code on each chunk concurrently. To take advantage of GPU hardware, computations need to be executed on the GPU in parallel.

Julia's multiple dispatch mechanism allows functions to be specialized for different types of data and to dispatch the computation to the GPU when appropriate. This is achieved through Julia's `GPUArrays` package, which provides a GPU-backed array type that can be used in place of regular Julia arrays.

When a function is called on a `GPUArray`, Julia's multiple dispatch mechanism can determine if a GPU version of the function is available, and if so, dispatch the computation to the GPU. This allows computations to be executed in parallel on the GPU, which can lead to significant performance gains over executing the same computation on the CPU.

Here's an example:

```julia
using CUDA

function add_matrices(a::AbstractArray, b::AbstractArray)
    return a + b
end

# generate CPU arrays
a = rand(1000, 1000)
b = rand(1000, 1000)

# call the function on CPU arrays
c = add_matrices(a, b)

# generate GPU arrays
a_gpu = CUDA.rand(1000, 1000)
b_gpu = CUDA.rand(1000, 1000)

# call the function on GPU arrays
c_gpu = add_matrices(a_gpu, b_gpu)

```



In this example, we define a function `add_matrices` that adds two arrays together. When called with `a_gpu` and `b_gpu`, Julia's multiple dispatch mechanism recognizes that a GPU version of the function is available and dispatches the computation to the GPU.

The resulting `c_gpu` array contains the result of the computation, which can be copied back to the CPU using the `Array` function.

In summary, multiple dispatch in Julia can be useful when it comes to GPU computing because it allows functions to be specialized for different types of data and to dispatch computations to the GPU when appropriate. This can lead to significant performance gains, especially when working with large arrays.

## Julia Integrates well with existing code

Calling C, Fortran, Python, and other libraries from  Julia is easy thanks to its first class support for interoperability.

More on that later on.

## 4. Productivity

- Code can be made very generic
  - For instance, code can be very effortlessly used for GPU computing
- Research script can be easily transformed into **packages**, directly available to the community

## 6. Composability of libraries and scientific ML
### Julia is an automatic differentiation pervasive language [(Innes et al., 2019)](http://arxiv.org/abs/1907.07587)

### The [SciML](https://sciml.ai) ecosystem

SciML is a composable open source software for scientific machine learning with differentiable programming.
   -  Read [this very cool paper](https://arxiv.org/abs/2001.04385) introducing the [SciML software ecosystem]
   - [Example using Deep Learning libraries in combination with ODE solvers](ode_solvers_with_DL.jl)

```julia
function lotka_volterra(u, p, t)
    weights_nn, α, δ = p
    🐰, 🦊 = u

    nn_res = neuralnet(🦊, 🐰, weights_nn)

    d🐰 = α * 🐰 - nn_res[1]
    d🦊 = nn_res[2]  - δ * 🦊

    return [d🐰, d🦊]
end
```

### Bayesian inference framework with deep learning

[Here is another cool example](https://turinglang.org/v0.24/tutorials/03-bayesian-neural-network/) of composability between [`Turing.jl`](https://turinglang.org/v0.24/) and `Flux.jl`

![](https://turinglang.org/v0.24/tutorials/figures/03_bayesian-neural-network_9_1.png)

## A few packages that I have developed

### [EvoId.jl](https://github.com/vboussange/EvoId.jl)
Evolutionary Individual based modelling, mathematically grounded. A user friendly package aimed at simulating the evolutionary dynamics of a population structured over a complex spatio-evolutionary structures.

### [HighDimPDE.jl](https://github.com/SciML/HighDimPDE.jl)
Solver for **highly dimensional, non-local, nonlinear PDEs**. It is integrated within the SciML ecosystem (see below). Try it out! &#128515; If you want to learn more about the algorithms implemented, check out my [research interests]({{site.url}}/research/#developping-numerical-schemes-for-solving-high-dimensional-non-local-nonlinear-pdes).

### [PiecewiseInference.jl](https://github.com/vboussange/PiecewiseInference.jl)
Suite for parameter inference and model selection with dynamical models characterised by complex dynamics.

### [ParametricModels.jl](https://github.com/vboussange/ParametricModels.jl)
Utilities for parametric and composite differential equation models.

### [EcoEvoModelZoo.jl](https://github.com/vboussange/EcoEvoModelZoo.jl)
A zoo of eco-evolutionary models with high fitness.

### [SciML](https://github.com/SciML/)
I am a member of the **SciML** organisation, an open source ecosystem for Scientific Machine Learning in the Julia programming language. On top of being the main author of **HighDimPDE.jl**, I actively participate in the development of other packages such as [DiffEqFlux.jl](https://github.com/SciML/DiffEqFlux.jl), a library to train differential equations with data.
